In [8]:
import pandas as pd

df = pd.read_csv("Churn_Modelling.csv")

In [9]:
import pickle

col_trans = None
with open("col_trans.pkl", "rb") as file:
    col_trans = pickle.load(file)

In [10]:
from sklearn.model_selection import train_test_split

X = df.drop("Exited", axis=1)
y = df["Exited"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45)

In [11]:
X_train.shape

(8000, 13)

In [12]:
X_train = col_trans.fit_transform(X_train)
X_test = col_trans.transform(X_test)

In [13]:
X_train.shape

(8000, 12)

In [14]:
X_train[0]

array([ 0.        ,  1.        ,  0.        ,  0.        , -0.54314547,
        2.0207228 , -1.73524323,  0.88733828, -0.91218445, -1.54081269,
       -1.02840321, -1.55151018])

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import datetime
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

In [16]:
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1], )),
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")
])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [21]:
# callbacks
log_dir = "logs/fit"

tensorboard_callback = TensorBoard(log_dir)
early_stopping_callback = EarlyStopping(monitor="accuracy", patience=7, restore_best_weights=True)

In [22]:
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=25,
    callbacks=[
        tensorboard_callback,
        early_stopping_callback
    ]
)

Epoch 1/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3067 - accuracy: 0.8748 - val_loss: 0.3452 - val_accuracy: 0.8540
Epoch 2/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3047 - accuracy: 0.8731 - val_loss: 0.3477 - val_accuracy: 0.8530
Epoch 3/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3047 - accuracy: 0.8733 - val_loss: 0.3466 - val_accuracy: 0.8555
Epoch 4/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3037 - accuracy: 0.8717 - val_loss: 0.3578 - val_accuracy: 0.8610
Epoch 5/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3042 - accuracy: 0.8764 - val_loss: 0.3471 - val_accuracy: 0.8570
Epoch 6/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3024 - accuracy: 0.8742 - val_loss: 0.3515 - val_accuracy: 0.8595
Epoch 7/25
250/250 [==============================] - 0s 2ms/step - loss: 0.3006 - accuracy: 0.8760 - val_loss: 0.3557 - val_accuracy: 0.8575
Epoch 

In [23]:
model.save("model.keras")